## Creating data for main results table

In [1]:
from __future__ import annotations

import pandas as pd

import mteb

mdl_names = [
    "sentence-transformers/all-MiniLM-L6-v2",
    "sentence-transformers/all-MiniLM-L12-v2",
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    "sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    "sentence-transformers/all-mpnet-base-v2",
    "sentence-transformers/LaBSE",
    "intfloat/multilingual-e5-large-instruct",
    "intfloat/e5-mistral-7b-instruct",
    "GritLM/GritLM-7B",
    "intfloat/multilingual-e5-small",
    "intfloat/multilingual-e5-base",
    "intfloat/multilingual-e5-large",
]
model_metas = [mteb.get_model_meta(name) for name in mdl_names]

/Users/au561649/Github/mteb/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def add_aggregate_columns(results):
    task_names = results.columns[2:]

    # convert to 100 scale
    results[task_names] = results[task_names] * 100

    borda = results[task_names].rank(ascending=True, method="min").sum(axis=1)
    results["Borda Count"] = borda
    results = results.sort_values("Borda Count", ascending=False)
    # borda str: 1 ({borda count}) 2 ({borda count}) 3 ({borda count}) ...
    results["Borda str"] = [
        f"{i + 1} ({int(borda_count)})"
        for i, borda_count in enumerate(results["Borda Count"].to_list())
    ]

    # add mean across tasks
    results["Mean"] = results[task_names].mean(axis=1)

    # add mean pr. task type
    task_types = [
        "BitextMining",
        "PairClassification",
        "Classification",
        "STS",
        "Retrieval",
        "MultilabelClassification",
        "Clustering",
        "Reranking",
    ]

    tasks = [mteb.get_task(name) for name in task_names]
    tasktype_to_tasks = {
        task_type: [t for t in tasks if t.metadata.type == task_type]
        for task_type in task_types
    }

    for task_type, tasks in tasktype_to_tasks.items():
        task_names = [t.metadata.name for t in tasks]
        results[f"Mean {task_type}"] = results[task_names].mean(axis=1)

    # add mean pr. task type
    cols = [f"Mean {task_type}" for task_type in task_types]
    results["mean pr. task type"] = results[cols].mean(axis=1)
    return results

In [3]:
mult_tasks = mteb.get_benchmark("MTEB(Indic)").tasks

# load task results for the specified models from mteb/results repository
mteb_results = mteb.load_results(
    models=model_metas,
    tasks=mult_tasks,
    download_latest=False,
)

In [4]:
mteb_results = mteb_results.join_revisions()

## Indic


In [5]:
mult_tasks = mteb.get_benchmark("MTEB(Indic)").tasks

# load task results for the specified models from mteb/results repository
mteb_results = mteb.load_results(
    models=model_metas,
    tasks=mult_tasks,
    download_latest=False,
)

mteb_results = mteb_results.join_revisions().filter_models()

# manual check that everything is there
# pd.DataFrame(mteb_results.get_scores()).to_csv("tmp.csv")

results = pd.DataFrame(mteb_results.get_scores())
results = add_aggregate_columns(results=results)


# create latex table
# column order
cols = [
    "model",
    "Borda str",
    "Mean",
    "mean pr. task type",
    "Mean BitextMining",
    "Mean PairClassification",
    "Mean Classification",
    "Mean STS",
    "Mean Retrieval",
    "Mean MultilabelClassification",
    "Mean Clustering",
    "Mean Reranking",
]

latex_df = results[cols]

/Users/au561649/Github/mteb/mteb/load_results/benchmark_results.py:120: UserWarning: Couldn't get scores for IndicGenBenchFloresBitextMining due to No splits had scores for the specified languages..
  warnings.warn(
/Users/au561649/Github/mteb/mteb/load_results/benchmark_results.py:120: UserWarning: Couldn't get scores for IndicLangClassification due to No splits had scores for the specified languages..
  warnings.warn(
/Users/au561649/Github/mteb/mteb/load_results/benchmark_results.py:120: UserWarning: Couldn't get scores for LinceMTBitextMining due to No splits had scores for the specified languages..
  warnings.warn(


In [6]:
results

,model,revision,BelebeleRetrieval,BengaliSentimentAnalysis,GujaratiNewsClassification,HindiDiscourseClassification,IN22ConvBitextMining,IN22GenBitextMining,IndicCrosslingualSTS,MTOPIntentClassification,...,Mean,Mean BitextMining,Mean PairClassification,Mean Classification,Mean STS,Mean Retrieval,Mean MultilabelClassification,Mean Clustering,Mean Reranking,mean pr. task type
4,intfloat/multilingual-e5-large-instruct,baa7be480a7de1539afce709c8f13f833a510e0a,73.754577,83.984927,87.518968,35.234375,71.873549,88.875417,53.688533,62.952996,...,70.186085,80.374483,76.314311,67.009249,53.688533,84.862788,NaN,51.671254,87.462063,71.626097
3,intfloat/multilingual-e5-large,4dc6d853a804b9c8886ede6dda8a073b7dc08a81,68.199269,83.070706,76.737481,38.740234,67.784590,87.696664,43.866162,59.198891,...,66.354869,77.740627,75.057575,64.659155,43.866162,82.604635,NaN,25.602675,85.970595,65.071632
2,intfloat/multilingual-e5-base,d13f1b27baf31030b7fd040960d60d909913633f,60.371654,79.643111,74.908953,39.038086,63.130747,85.290012,41.113890,54.049160,...,64.571532,74.210379,72.795771,63.753851,41.113890,77.842327,NaN,24.607904,83.761455,62.583654
5,intfloat/multilingual-e5-small,e4ce9877abf3edfe10b0d82785e83bdcb973e22e,58.193538,83.430444,74.393020,39.335938,62.739174,84.663212,40.761080,52.118503,...,64.720156,73.701193,73.795104,63.782196,40.761080,76.817269,NaN,29.054088,84.369577,63.182930
0,GritLM/GritLM-7B,13f00a0e36500c80ce12870ea513846a066004af,70.063654,72.100459,69.855842,37.089844,42.137574,74.667530,27.245620,63.660123,...,60.204173,58.402552,67.838170,60.043953,27.245620,79.496827,NaN,27.978331,84.695146,57.957228
1,intfloat/e5-mistral-7b-instruct,07163b72af1488142a360786df853f237b1a3ca1,66.289423,72.074816,73.088012,32.006836,44.302781,73.799887,22.981297,59.232093,...,60.022827,59.051334,72.951099,59.563869,22.981297,77.266212,NaN,32.702546,84.420079,58.419491
6,sentence-transformers/LaBSE,e34fab64a3011d2176c99545a93d5cbddc9a91b7,47.515538,80.415418,76.358118,38.398438,63.459252,84.668840,52.758018,62.863971,...,61.855117,74.064046,64.583366,61.906474,52.758018,64.334769,NaN,21.105169,78.980476,59.676046
11,sentence-transformers/paraphrase-multilingual-...,79f2382ceacceacdf38563d7c5d16b9ff8d725d6,36.101692,74.882594,81.934750,38.691406,33.588414,54.802443,34.096874,61.699834,...,58.502887,44.195428,82.036142,61.943266,34.096874,57.910346,NaN,32.061697,74.332275,55.225147
10,sentence-transformers/paraphrase-multilingual-...,bf3bf13ab40c3157080a7ab344c831b9ad18b5eb,19.395077,60.613522,76.874052,37.431641,11.867235,18.709649,19.788971,59.196945,...,49.672632,15.288442,77.849520,57.645424,19.788971,48.779038,NaN,16.675403,59.258690,42.183641
9,sentence-transformers/all-mpnet-base-v2,84f2bcc00d77236f9e89c8a360a00fb1139bf47d,9.627615,54.188273,44.104704,34.316406,2.110360,5.353099,-2.509332,18.150352,...,33.630293,3.731730,52.634533,45.224600,-2.509332,12.853808,NaN,4.012625,42.601772,22.649962


In [7]:
latex_df

,model,Borda str,Mean,mean pr. task type,Mean BitextMining,Mean PairClassification,Mean Classification,Mean STS,Mean Retrieval,Mean MultilabelClassification,Mean Clustering,Mean Reranking
4,intfloat/multilingual-e5-large-instruct,1 (209),70.186085,71.626097,80.374483,76.314311,67.009249,53.688533,84.862788,NaN,51.671254,87.462063
3,intfloat/multilingual-e5-large,2 (188),66.354869,65.071632,77.740627,75.057575,64.659155,43.866162,82.604635,NaN,25.602675,85.970595
2,intfloat/multilingual-e5-base,3 (173),64.571532,62.583654,74.210379,72.795771,63.753851,41.113890,77.842327,NaN,24.607904,83.761455
5,intfloat/multilingual-e5-small,4 (164),64.720156,63.182930,73.701193,73.795104,63.782196,40.761080,76.817269,NaN,29.054088,84.369577
0,GritLM/GritLM-7B,5 (151),60.204173,57.957228,58.402552,67.838170,60.043953,27.245620,79.496827,NaN,27.978331,84.695146
1,intfloat/e5-mistral-7b-instruct,6 (144),60.022827,58.419491,59.051334,72.951099,59.563869,22.981297,77.266212,NaN,32.702546,84.420079
6,sentence-transformers/LaBSE,7 (139),61.855117,59.676046,74.064046,64.583366,61.906474,52.758018,64.334769,NaN,21.105169,78.980476
11,sentence-transformers/paraphrase-multilingual-...,8 (137),58.502887,55.225147,44.195428,82.036142,61.943266,34.096874,57.910346,NaN,32.061697,74.332275
10,sentence-transformers/paraphrase-multilingual-...,9 (98),49.672632,42.183641,15.288442,77.849520,57.645424,19.788971,48.779038,NaN,16.675403,59.258690
9,sentence-transformers/all-mpnet-base-v2,10 (68),33.630293,22.649962,3.731730,52.634533,45.224600,-2.509332,12.853808,NaN,4.012625,42.601772


In [8]:
print(latex_df.to_latex(index=False, float_format="%.1f"))

\begin{tabular}{llrrrrrrrrrr}
\toprule
model & Borda str & Mean & mean pr. task type & Mean BitextMining & Mean PairClassification & Mean Classification & Mean STS & Mean Retrieval & Mean MultilabelClassification & Mean Clustering & Mean Reranking \\
\midrule
intfloat/multilingual-e5-large-instruct & 1 (209) & 70.2 & 71.6 & 80.4 & 76.3 & 67.0 & 53.7 & 84.9 & NaN & 51.7 & 87.5 \\
intfloat/multilingual-e5-large & 2 (188) & 66.4 & 65.1 & 77.7 & 75.1 & 64.7 & 43.9 & 82.6 & NaN & 25.6 & 86.0 \\
intfloat/multilingual-e5-base & 3 (173) & 64.6 & 62.6 & 74.2 & 72.8 & 63.8 & 41.1 & 77.8 & NaN & 24.6 & 83.8 \\
intfloat/multilingual-e5-small & 4 (164) & 64.7 & 63.2 & 73.7 & 73.8 & 63.8 & 40.8 & 76.8 & NaN & 29.1 & 84.4 \\
GritLM/GritLM-7B & 5 (151) & 60.2 & 58.0 & 58.4 & 67.8 & 60.0 & 27.2 & 79.5 & NaN & 28.0 & 84.7 \\
intfloat/e5-mistral-7b-instruct & 6 (144) & 60.0 & 58.4 & 59.1 & 73.0 & 59.6 & 23.0 & 77.3 & NaN & 32.7 & 84.4 \\
sentence-transformers/LaBSE & 7 (139) & 61.9 & 59.7 & 74.1 & 64.6 &

## Europe

In [9]:
mult_tasks = mteb.get_benchmark("MTEB(Europe)").tasks

# load task results for the specified models from mteb/results repository
mteb_results = mteb.load_results(
    models=model_metas,
    tasks=mult_tasks,
    download_latest=False,
)

mteb_results = mteb_results.join_revisions().filter_models()

# manual check that everything is there
pd.DataFrame(mteb_results.get_scores()).to_csv("tmp.csv")

results = pd.DataFrame(mteb_results.get_scores())
results = add_aggregate_columns(results=results)


# create latex table
# column order
cols = [
    "model",
    "Borda str",
    "Mean",
    "mean pr. task type",
    "Mean BitextMining",
    "Mean PairClassification",
    "Mean Classification",
    "Mean STS",
    "Mean Retrieval",
    "Mean MultilabelClassification",
    "Mean Clustering",
    "Mean Reranking",
]

latex_df = results[cols]

/Users/au561649/Github/mteb/mteb/load_results/benchmark_results.py:120: UserWarning: Couldn't get scores for NordicLangClassification due to No splits had scores for the specified languages..
  warnings.warn(


In [10]:
results

,model,revision,AlloProfClusteringS2S.v2,AlloprofReranking,AlloprofRetrieval,AmazonCounterfactualClassification,ArguAna,BUCC.v2,BelebeleRetrieval,BibleNLPBitextMining,...,Mean,Mean BitextMining,Mean PairClassification,Mean Classification,Mean STS,Mean Retrieval,Mean MultilabelClassification,Mean Clustering,Mean Reranking,mean pr. task type
0,GritLM/GritLM-7B,13f00a0e36500c80ce12870ea513846a066004af,56.411821,77.926160,55.422,78.452787,63.171,99.502429,91.393110,97.337705,...,62.970819,90.420705,89.939440,64.736896,76.050247,57.105263,17.551200,45.281105,60.269644,62.669312
4,intfloat/multilingual-e5-large-instruct,baa7be480a7de1539afce709c8f13f833a510e0a,56.465655,74.677730,52.118,67.619802,58.476,99.473836,92.240123,97.916667,...,62.192929,90.383846,89.985879,63.241319,77.428657,54.802566,17.265889,46.895583,58.420108,62.302981
1,intfloat/e5-mistral-7b-instruct,07163b72af1488142a360786df853f237b1a3ca1,57.112000,78.317664,54.619,73.901466,61.653,99.384579,88.393877,96.731306,...,61.729119,89.580187,91.154182,62.946715,76.481287,53.644095,15.464545,46.473328,59.815300,61.944955
3,intfloat/multilingual-e5-large,4dc6d853a804b9c8886ede6dda8a073b7dc08a81,35.150764,69.442880,39.341,75.116732,54.357,99.022547,92.728438,94.571508,...,58.491543,84.456404,88.753477,60.385125,75.759056,50.814358,14.980040,38.235917,55.910580,58.661870
2,intfloat/multilingual-e5-base,d13f1b27baf31030b7fd040960d60d909913633f,34.113190,65.897152,34.447,75.098079,44.206,98.699878,87.649863,94.283655,...,57.186738,84.110742,87.352572,57.854090,73.669498,50.201567,14.862272,38.160560,53.854468,57.508221
11,sentence-transformers/paraphrase-multilingual-...,79f2382ceacceacdf38563d7c5d16b9ff8d725d6,41.806274,67.204322,30.799,73.983687,48.908,98.330171,79.738726,95.205078,...,54.410453,79.468659,90.725512,56.599349,74.253507,41.160312,6.897841,35.783209,52.336680,54.653134
5,intfloat/multilingual-e5-small,e4ce9877abf3edfe10b0d82785e83bdcb973e22e,35.393261,64.410023,27.380,71.746625,39.088,96.352984,82.592438,85.072037,...,55.038105,80.948799,86.374052,56.108885,71.636076,46.072279,13.967448,36.498080,54.108918,55.714317
6,sentence-transformers/LaBSE,e34fab64a3011d2176c99545a93d5cbddc9a91b7,30.208930,55.374766,19.775,74.485455,34.178,99.189150,72.629699,97.470989,...,51.843718,88.779284,85.180182,55.100932,65.683818,34.351709,16.298114,34.253079,48.660051,53.538396
10,sentence-transformers/paraphrase-multilingual-...,bf3bf13ab40c3157080a7ab344c831b9ad18b5eb,40.451213,62.424382,26.634,69.772668,44.878,97.167386,74.561219,93.669550,...,51.731912,76.988968,88.925092,52.678430,72.536393,37.598783,5.687953,34.443163,50.198119,52.382113
9,sentence-transformers/all-mpnet-base-v2,84f2bcc00d77236f9e89c8a360a00fb1139bf47d,35.215154,69.630056,34.270,62.193713,46.521,26.357644,39.288041,6.588554,...,44.687410,29.807469,80.519579,49.249220,63.883592,37.307847,10.872124,36.190054,49.608487,44.679796


In [11]:
latex_df

,model,Borda str,Mean,mean pr. task type,Mean BitextMining,Mean PairClassification,Mean Classification,Mean STS,Mean Retrieval,Mean MultilabelClassification,Mean Clustering,Mean Reranking
0,GritLM/GritLM-7B,1 (757),62.970819,62.669312,90.420705,89.939440,64.736896,76.050247,57.105263,17.551200,45.281105,60.269644
4,intfloat/multilingual-e5-large-instruct,2 (732),62.192929,62.302981,90.383846,89.985879,63.241319,77.428657,54.802566,17.265889,46.895583,58.420108
1,intfloat/e5-mistral-7b-instruct,3 (725),61.729119,61.944955,89.580187,91.154182,62.946715,76.481287,53.644095,15.464545,46.473328,59.815300
3,intfloat/multilingual-e5-large,4 (586),58.491543,58.661870,84.456404,88.753477,60.385125,75.759056,50.814358,14.980040,38.235917,55.910580
2,intfloat/multilingual-e5-base,5 (499),57.186738,57.508221,84.110742,87.352572,57.854090,73.669498,50.201567,14.862272,38.160560,53.854468
11,sentence-transformers/paraphrase-multilingual-...,6 (463),54.410453,54.653134,79.468659,90.725512,56.599349,74.253507,41.160312,6.897841,35.783209,52.336680
5,intfloat/multilingual-e5-small,7 (399),55.038105,55.714317,80.948799,86.374052,56.108885,71.636076,46.072279,13.967448,36.498080,54.108918
6,sentence-transformers/LaBSE,8 (358),51.843718,53.538396,88.779284,85.180182,55.100932,65.683818,34.351709,16.298114,34.253079,48.660051
10,sentence-transformers/paraphrase-multilingual-...,9 (328),51.731912,52.382113,76.988968,88.925092,52.678430,72.536393,37.598783,5.687953,34.443163,50.198119
9,sentence-transformers/all-mpnet-base-v2,10 (310),44.687410,44.679796,29.807469,80.519579,49.249220,63.883592,37.307847,10.872124,36.190054,49.608487


In [12]:
print(latex_df.to_latex(index=False, float_format="%.1f"))

\begin{tabular}{llrrrrrrrrrr}
\toprule
model & Borda str & Mean & mean pr. task type & Mean BitextMining & Mean PairClassification & Mean Classification & Mean STS & Mean Retrieval & Mean MultilabelClassification & Mean Clustering & Mean Reranking \\
\midrule
GritLM/GritLM-7B & 1 (757) & 63.0 & 62.7 & 90.4 & 89.9 & 64.7 & 76.1 & 57.1 & 17.6 & 45.3 & 60.3 \\
intfloat/multilingual-e5-large-instruct & 2 (732) & 62.2 & 62.3 & 90.4 & 90.0 & 63.2 & 77.4 & 54.8 & 17.3 & 46.9 & 58.4 \\
intfloat/e5-mistral-7b-instruct & 3 (725) & 61.7 & 61.9 & 89.6 & 91.2 & 62.9 & 76.5 & 53.6 & 15.5 & 46.5 & 59.8 \\
intfloat/multilingual-e5-large & 4 (586) & 58.5 & 58.7 & 84.5 & 88.8 & 60.4 & 75.8 & 50.8 & 15.0 & 38.2 & 55.9 \\
intfloat/multilingual-e5-base & 5 (499) & 57.2 & 57.5 & 84.1 & 87.4 & 57.9 & 73.7 & 50.2 & 14.9 & 38.2 & 53.9 \\
sentence-transformers/paraphrase-multilingual-mpnet-base-v2 & 6 (463) & 54.4 & 54.7 & 79.5 & 90.7 & 56.6 & 74.3 & 41.2 & 6.9 & 35.8 & 52.3 \\
intfloat/multilingual-e5-small & 

## Multilingual

In [13]:
mult_tasks = mteb.get_benchmark("MTEB(Multilingual)").tasks

# load task results for the specified models from mteb/results repository
mteb_results = mteb.load_results(
    models=model_metas,
    tasks=mult_tasks,
    download_latest=False,
)

mteb_results = mteb_results.join_revisions().filter_models()

# manual check that everything is there
pd.DataFrame(mteb_results.get_scores()).to_csv("tmp.csv")

results = pd.DataFrame(mteb_results.get_scores())
results = add_aggregate_columns(results=results)


# create latex table
# column order
cols = [
    "model",
    "Borda str",
    "Mean",
    "mean pr. task type",
    "Mean BitextMining",
    "Mean PairClassification",
    "Mean Classification",
    "Mean STS",
    "Mean Retrieval",
    "Mean MultilabelClassification",
    "Mean Clustering",
    "Mean Reranking",
]

latex_df = results[cols]

/var/folders/bq/3m2kv2_535q0c9ld2jmmz774yj4nph/T/ipykernel_22431/597027223.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  results["Borda Count"] = borda
/var/folders/bq/3m2kv2_535q0c9ld2jmmz774yj4nph/T/ipykernel_22431/597027223.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  results["Borda str"] = [
/var/folders/bq/3m2kv2_535q0c9ld2jmmz774yj4nph/T/ipykernel_22431/597027223.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performa

In [14]:
results

,model,revision,AILAStatutes,AfriSentiClassification,AlloProfClusteringS2S.v2,AlloprofReranking,AmazonCounterfactualClassification,ArXivHierarchicalClusteringP2P,ArXivHierarchicalClusteringS2S,ArguAna,...,Mean,Mean BitextMining,Mean PairClassification,Mean Classification,Mean STS,Mean Retrieval,Mean MultilabelClassification,Mean Clustering,Mean Reranking,mean pr. task type
4,intfloat/multilingual-e5-large-instruct,baa7be480a7de1539afce709c8f13f833a510e0a,29.659,45.387432,56.465655,74.677730,68.606356,62.534994,61.284058,58.476,...,63.227169,80.126470,80.863584,64.942144,76.814671,57.116686,22.913504,51.538016,62.613273,62.116044
0,GritLM/GritLM-7B,13f00a0e36500c80ce12870ea513846a066004af,41.800,45.078589,56.411821,77.926160,79.296512,59.760046,62.283243,63.171,...,60.930855,70.531726,79.944411,61.830247,73.327960,58.306711,22.773771,50.482520,63.778754,60.122013
1,intfloat/e5-mistral-7b-instruct,07163b72af1488142a360786df853f237b1a3ca1,34.535,44.476335,57.112000,78.317664,73.555839,65.283735,61.278123,61.653,...,60.279746,70.579905,81.122211,60.314295,74.021600,55.750101,22.196821,51.390095,63.819183,59.899276
3,intfloat/multilingual-e5-large,4dc6d853a804b9c8886ede6dda8a073b7dc08a81,20.842,45.500507,35.150764,69.442880,76.163514,55.572114,56.212217,54.357,...,58.570572,71.666250,79.028390,59.916975,73.488372,54.111178,21.302373,42.923755,62.840466,58.159720
2,intfloat/multilingual-e5-base,d13f1b27baf31030b7fd040960d60d909913633f,20.371,43.802315,34.113190,65.897152,74.334014,56.683137,56.115056,44.206,...,57.012959,69.437945,77.154385,58.205718,71.444249,52.721823,20.162060,42.674461,60.176360,56.497125
11,sentence-transformers/paraphrase-multilingual-...,79f2382ceacceacdf38563d7c5d16b9ff8d725d6,22.236,42.445471,41.806274,67.204322,72.766528,55.342767,55.160328,48.908,...,52.005259,52.062937,81.154392,55.064354,69.661040,39.757752,16.398034,41.080665,53.374677,51.069231
5,intfloat/multilingual-e5-small,e4ce9877abf3edfe10b0d82785e83bdcb973e22e,19.011,42.358118,35.393261,64.410023,69.163655,54.276235,54.198749,39.088,...,55.456701,67.472922,76.329053,56.500928,70.360817,49.345019,19.096430,41.735446,60.391010,55.153953
6,sentence-transformers/LaBSE,e34fab64a3011d2176c99545a93d5cbddc9a91b7,16.717,43.170657,30.208930,55.374766,74.987910,53.442702,49.986064,34.178,...,52.100056,76.351008,75.969075,54.600844,65.349763,33.169113,20.122117,39.159195,50.197562,51.864835
10,sentence-transformers/paraphrase-multilingual-...,bf3bf13ab40c3157080a7ab344c831b9ad18b5eb,20.525,37.672740,40.451213,62.424382,68.075646,53.617944,52.245736,44.878,...,48.781520,44.563390,78.993229,51.656889,66.581953,36.614971,14.930329,39.337371,50.972387,47.956315
9,sentence-transformers/all-mpnet-base-v2,84f2bcc00d77236f9e89c8a360a00fb1139bf47d,21.275,37.267741,35.215154,69.630056,61.846281,61.473392,56.459312,46.521,...,42.470049,21.161317,70.893513,46.985488,57.599660,32.808557,16.280508,40.765913,42.234410,41.091171


In [15]:
latex_df

,model,Borda str,Mean,mean pr. task type,Mean BitextMining,Mean PairClassification,Mean Classification,Mean STS,Mean Retrieval,Mean MultilabelClassification,Mean Clustering,Mean Reranking
4,intfloat/multilingual-e5-large-instruct,1 (1375),63.227169,62.116044,80.126470,80.863584,64.942144,76.814671,57.116686,22.913504,51.538016,62.613273
0,GritLM/GritLM-7B,2 (1258),60.930855,60.122013,70.531726,79.944411,61.830247,73.327960,58.306711,22.773771,50.482520,63.778754
1,intfloat/e5-mistral-7b-instruct,3 (1233),60.279746,59.899276,70.579905,81.122211,60.314295,74.021600,55.750101,22.196821,51.390095,63.819183
3,intfloat/multilingual-e5-large,4 (1109),58.570572,58.159720,71.666250,79.028390,59.916975,73.488372,54.111178,21.302373,42.923755,62.840466
2,intfloat/multilingual-e5-base,5 (944),57.012959,56.497125,69.437945,77.154385,58.205718,71.444249,52.721823,20.162060,42.674461,60.176360
11,sentence-transformers/paraphrase-multilingual-...,6 (830),52.005259,51.069231,52.062937,81.154392,55.064354,69.661040,39.757752,16.398034,41.080665,53.374677
5,intfloat/multilingual-e5-small,7 (784),55.456701,55.153953,67.472922,76.329053,56.500928,70.360817,49.345019,19.096430,41.735446,60.391010
6,sentence-transformers/LaBSE,8 (719),52.100056,51.864835,76.351008,75.969075,54.600844,65.349763,33.169113,20.122117,39.159195,50.197562
10,sentence-transformers/paraphrase-multilingual-...,9 (603),48.781520,47.956315,44.563390,78.993229,51.656889,66.581953,36.614971,14.930329,39.337371,50.972387
9,sentence-transformers/all-mpnet-base-v2,10 (526),42.470049,41.091171,21.161317,70.893513,46.985488,57.599660,32.808557,16.280508,40.765913,42.234410


In [16]:
print(latex_df.to_latex(index=False, float_format="%.1f"))

\begin{tabular}{llrrrrrrrrrr}
\toprule
model & Borda str & Mean & mean pr. task type & Mean BitextMining & Mean PairClassification & Mean Classification & Mean STS & Mean Retrieval & Mean MultilabelClassification & Mean Clustering & Mean Reranking \\
\midrule
intfloat/multilingual-e5-large-instruct & 1 (1375) & 63.2 & 62.1 & 80.1 & 80.9 & 64.9 & 76.8 & 57.1 & 22.9 & 51.5 & 62.6 \\
GritLM/GritLM-7B & 2 (1258) & 60.9 & 60.1 & 70.5 & 79.9 & 61.8 & 73.3 & 58.3 & 22.8 & 50.5 & 63.8 \\
intfloat/e5-mistral-7b-instruct & 3 (1233) & 60.3 & 59.9 & 70.6 & 81.1 & 60.3 & 74.0 & 55.8 & 22.2 & 51.4 & 63.8 \\
intfloat/multilingual-e5-large & 4 (1109) & 58.6 & 58.2 & 71.7 & 79.0 & 59.9 & 73.5 & 54.1 & 21.3 & 42.9 & 62.8 \\
intfloat/multilingual-e5-base & 5 (944) & 57.0 & 56.5 & 69.4 & 77.2 & 58.2 & 71.4 & 52.7 & 20.2 & 42.7 & 60.2 \\
sentence-transformers/paraphrase-multilingual-mpnet-base-v2 & 6 (830) & 52.0 & 51.1 & 52.1 & 81.2 & 55.1 & 69.7 & 39.8 & 16.4 & 41.1 & 53.4 \\
intfloat/multilingual-e5-sma